In [1]:
import json
import numpy as np

In [2]:
def create_scale_mod(key, intervals):

    note = key

    scale = [note]

    for interval in intervals:

        note += interval
        scale.append(note%12)
    
    return scale


def create_scale(key, intervals):

    note = key

    scale = [note]

    for interval in intervals:

        note += interval
        scale.append(note)
    
    return scale


def map_to_key(scale, notes):
    
    return [notes[val] for val in scale]

In [4]:
with open('../data/metadata/note_frequencies.json','r') as infile:
    note_frequencies = json.load(infile)
    
notes = [note.split('/')[0][:-1] for note in list(note_frequencies.keys())[:12]]

In [5]:
print(notes)

['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']


In [6]:
major_intervals = [2,2,1,2,2,2]
minor_intervals = [2,1,2,2,1,2]

In [7]:
sub_frequencies = list(note_frequencies.values())[12:24]
bass0_frequencies = list(note_frequencies.values())[24:36]
bass1_frequencies = list(note_frequencies.values())[36:48]
low_mid_frequencies = list(note_frequencies.values())[48:60]

print('Note\tSub\tBass0\tBass1\tLow Mid')
for i in range(12):
    
    print("{}\t{}\t{}\t{}\t{}".format(notes[i],sub_frequencies[i], bass0_frequencies[i], bass1_frequencies[i],low_mid_frequencies[i]))
       
        
bass_frequencies = bass0_frequencies+bass1_frequencies

Note	Sub	Bass0	Bass1	Low Mid
C	32.70	65.41	130.81	261.63
C#	34.65	69.30	138.59	277.18
D	36.71	73.42	146.83	293.66
D#	38.89	77.78	155.56	311.13
E	41.20	82.41	164.81	329.63
F	43.65	87.31	174.61	349.23
F#	46.25	92.50	185.00	369.99
G	49.00	98.00	196.00	392.00
G#	51.91	103.83	207.65	415.30
A	55.00	110.00	220.00	440.00
A#	58.27	116.54	233.08	466.16
B	61.74	123.47	246.94	493.88


In [11]:
sorted([float(sub_frequencies[val]) for val in minor_scale_mod12])

[34.65, 36.71, 41.2, 46.25, 49.0, 55.0, 61.74]

In [25]:
scales = {n: {m: {k: {} for k in ['notes', 'frequencies', 'out_of_scale']} for m in['min','maj']} for n in notes}

for note in range(12):
    
    minor_scale = create_scale(note, minor_intervals)
    minor_scale_mod12 = [val%12 for val in minor_scale]
    notes_in_minor_scale = map_to_key(minor_scale_mod12, notes)
    out_scale_indices = [(idx+1)%12 for idx in minor_scale_mod12 if (idx+1)%12 not in minor_scale_mod12]
    notes_out_minor_scale = map_to_key(out_scale_indices, notes)
    
    scales[notes[note]]['min']['notes'] = notes_in_minor_scale
    scales[notes[note]]['min']['frequencies']['0'] =  sorted([float(sub_frequencies[val]) for val in minor_scale_mod12])
    scales[notes[note]]['min']['frequencies']['1'] =  sorted([float(bass0_frequencies[val]) for val in minor_scale_mod12])
    scales[notes[note]]['min']['frequencies']['2'] =  sorted([float(bass1_frequencies[val]) for val in minor_scale_mod12])
    
    f = [float(bass0_frequencies[val]) for val in out_scale_indices] + [float(bass1_frequencies[val]) for val in out_scale_indices]
    
    scales[notes[note]]['min']['out_of_scale']['notes'] = notes_out_minor_scale
    scales[notes[note]]['min']['out_of_scale']['frequencies'] =  sorted(f)
    
    
    major_scale = create_scale(note, major_intervals)
    major_scale_mod12 = [val%12 for val in major_scale]
    notes_in_major_scale = map_to_key(major_scale_mod12, notes)
    out_scale_indices = [(idx+1)%12 for idx in major_scale_mod12 if (idx+1)%12 not in major_scale_mod12]
    notes_out_major_scale = map_to_key(out_scale_indices, notes)
    
    scales[notes[note]]['maj']['notes'] = notes_in_major_scale
    scales[notes[note]]['maj']['frequencies']['0'] = sorted([float(sub_frequencies[val]) for val in major_scale_mod12])
    scales[notes[note]]['maj']['frequencies']['1'] = sorted([float(bass0_frequencies[val]) for val in major_scale_mod12])
    
    f = [float(bass0_frequencies[val]) for val in out_scale_indices] + [float(bass1_frequencies[val]) for val in out_scale_indices]
    
    scales[notes[note]]['maj']['out_of_scale']['notes'] = notes_out_major_scale
    scales[notes[note]]['maj']['out_of_scale']['frequencies'] =  sorted(f)

In [27]:
with open('../data/metadata/scales_frequencies.json', 'w') as of:
    json.dump(scales, of, indent=4)

In [ ]:
freq_array = np.array(list(scales['F#']['maj']['frequencies'].values()))

In [ ]:
min_freq_dict = {n: {m: {} for m in ['major','minor']} for n in notes}

a = 100
b = 100
c = 100

for key in scales.keys():
    
    for typ in scales[key].keys():
        
        freq_array = np.array(list(scales[key][typ]['frequencies'].values()))

        min_freq_difference = np.min(np.diff(freq_array,1,axis=1),axis=1)
        
        a = min(a,min_freq_difference[0])
        b = min(b,min_freq_difference[1])
        c = min(c,min_freq_difference[2])
        
        min_freq_dict[key][typ] = min_freq_difference
        
        
print("Lowest frequency differences for each band in all key/scale combinations: {:.1f}, {:.1f}, {:.1f}".format(a,b,c))